https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_10_and_11.ipynb

some deviations from the source code because i dont wanna pay for embeddings from openai, or hit openai models. All openAI integration is replaced with ollama.

I also removed langsmith integration. don't think it's needed. just a frontend for LLM debugging which i can achieve with `langchain.debug = True`

For query structuring i'll deviate quite heavily from the notebook above as the methods applied are quite trivial to implement, will attempt to implement a custom example myself.

In [1]:
import langchain 
langchain.debug = True 

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic.types import confloat
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from typing import Optional, Literal
from langchain_ollama.chat_models import ChatOllama

In [28]:
db_structured_prompt_template = """You are an expert at converting the users questions into database queries.
You have access to a database of restaurants. Given a search query in natural language, return a structured output for use in database search. 

This is the pydantic definition of the search query:
    cuisine: Literal['french', 'chinese', 'mexican', 'thai', 'indian', 'english', 'african', 'na'] = Field(
        ..., # ... means a mandatory field
        description="The type of cuisine the restaurant should serve: 'french', 'chinese', 'mexican', 'thai', 'indian', 'english', 'african', 'na'"
    )
    ratings: Optional[confloat(ge=1, le=5)] = Field(
        None,
        description='The ratings/stars that restaurant should have from 1 to 5, with 5 being the best.'
    ),
    price_point: Optional[Literal['$', '$$', '$$$', '$$$$', '$$$$$']] = Field(
        None,
        description="The price point of the restaurant, as '$', '$$', '$$$', '$$$$', '$$$$$' with '$$$$$' being the most expensive."
    ),
    reason: str = Field(
        ...,
        description="How the user question related to the cuisine"
    )

user question:
{user_question}
"""

db_structured_prompt = ChatPromptTemplate.from_template(db_structured_prompt_template)

In [29]:
class RestaurantSearch(BaseModel):
    cuisine: Literal['french', 'chinese', 'mexican', 'thai', 'indian', 'english', 'african', 'na'] = Field(
        ..., # ... means a mandatory field
        description="The type of cuisine the restaurant should serve: 'french', 'chinese', 'mexican', 'thai', 'indian', 'english', 'african', 'na'"
    )
    ratings: Optional[confloat(ge=1, le=5)] = Field(
        None,
        description='The ratings/stars that restaurant should have from 1 to 5, with 5 being the best.'
    ),
    price_point: Optional[Literal['$', '$$', '$$$', '$$$$', '$$$$$']] = Field(
        None,
        description="The price point of the restaurant, as '$', '$$', '$$$', '$$$$', '$$$$$' with '$$$$$' being the most expensive."
    ),
    reason: str = Field(
        ...,
        description="How the user question related to the cuisine"
    )

rest_structured_llm = ChatOllama(
    model="llama3.2:3b-instruct-q5_K_M",
    temperature=0,
    format=RestaurantSearch.model_json_schema()
)

/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=False, default=None, description='The ratings/stars that restaurant should have from 1 to 5, with 5 being the best.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=False, default=None, description="The price point of the restaurant, as '$', '$$', '$$$', '$$$$', '$$$$$' with '$$$$$' being the most expensive."),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [30]:
chain = db_structured_prompt | rest_structured_llm | JsonOutputParser()

print(chain.invoke({'user_question': "cheap tacos with >3.5 star"}))

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "user_question": "cheap tacos with >3.5 star"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "user_question": "cheap tacos with >3.5 star"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an expert at converting the users questions into database queries.\nYou have access to a database of restaurants. Given a search query in natural language, return a structured output for use in database search. \n\nThis is the pydantic definition of the search query:\n    cuisine: Literal['french', 'chinese', 'mexican', 'thai', 'indian', 'english', 'african', 'na'] = Field(\n        ..., # ... means a mandatory field\n        description=\"The type of cuisine the restaurant should serve: 'fre

In [31]:
print(chain.invoke({'user_question': "value for money tacos with >3.5 star"}))

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "user_question": "value for money tacos with >3.5 star"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "user_question": "value for money tacos with >3.5 star"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an expert at converting the users questions into database queries.\nYou have access to a database of restaurants. Given a search query in natural language, return a structured output for use in database search. \n\nThis is the pydantic definition of the search query:\n    cuisine: Literal['french', 'chinese', 'mexican', 'thai', 'indian', 'english', 'african', 'na'] = Field(\n        ..., # ... means a mandatory field\n        description=\"The type of cuisine the restauran

In [33]:
print(chain.invoke({'user_question': "upscale restaurant with har cheong gai with more than 4.2 star"}))

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "user_question": "upscale restaurant with har cheong gai with more than 4.2 star"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "user_question": "upscale restaurant with har cheong gai with more than 4.2 star"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an expert at converting the users questions into database queries.\nYou have access to a database of restaurants. Given a search query in natural language, return a structured output for use in database search. \n\nThis is the pydantic definition of the search query:\n    cuisine: Literal['french', 'chinese', 'mexican', 'thai', 'indian', 'english', 'african', 'na'] = Field(\n        ..., # ... means a mandatory field\n   